In [1]:
%cd /kaggle/working

/kaggle/working


In [2]:
import numpy as np
import polars as pl

In [163]:
test_info_df = pl.read_parquet("output/preprocess/make_hack_map/base/test_info.parquet")
test_old_info_df = pl.read_parquet(
    "output/preprocess/make_hack_map/base/test_old_info.parquet"
)

In [164]:
test_info_df = test_info_df.with_columns(
    (
        pl.datetime(2000, pl.col("month"), pl.col("day"), 0, 0, 0)
        + pl.duration(seconds=pl.col("seconds"))
    ).alias("no_year_timestamp")
).sort("no_year_timestamp")
test_info_df.head()

sample_id,location,month,day,seconds,tick,no_year_timestamp
str,u32,i64,i64,i64,i64,datetime[μs]
"""test_100324""",234,1,1,0,0,2000-01-01 00:00:00
"""test_101381""",46,1,1,0,0,2000-01-01 00:00:00
"""test_102391""",179,1,1,0,0,2000-01-01 00:00:00
"""test_103808""",66,1,1,0,0,2000-01-01 00:00:00
"""test_107667""",151,1,1,0,0,2000-01-01 00:00:00


In [165]:
test_old_info_df = (
    test_old_info_df.with_row_index()
    .with_columns(
        pl.when(pl.col("index") <= 282239)
        .then(pl.lit(9))
        .when(pl.col("index") <= 618623)
        .then(pl.lit(10))
        .otherwise(pl.lit(11))
        .alias("year")
    )
    .with_columns(
        (
            pl.datetime(pl.col("year"), pl.col("month"), pl.col("day"), 0, 0, 0)
            + pl.duration(seconds=pl.col("seconds"))
        ).alias("timestamp")
    )
    .with_columns(
        (
            pl.datetime(2000, pl.col("month"), pl.col("day"), 0, 0, 0)
            + pl.duration(seconds=pl.col("seconds"))
        ).alias("no_year_timestamp")
    )
    .sort("timestamp")
)
test_old_info_df.head()

index,sample_id,location,month,day,seconds,tick,year,timestamp,no_year_timestamp
u32,str,u32,i64,i64,i64,i64,i32,datetime[μs],datetime[μs]
0,"""test_169651""",0,3,1,0,0,9,0009-03-01 00:00:00,2000-03-01 00:00:00
1,"""test_524862""",1,3,1,0,0,9,0009-03-01 00:00:00,2000-03-01 00:00:00
2,"""test_634129""",2,3,1,0,0,9,0009-03-01 00:00:00,2000-03-01 00:00:00
3,"""test_403572""",3,3,1,0,0,9,0009-03-01 00:00:00,2000-03-01 00:00:00
4,"""test_484578""",4,3,1,0,0,9,0009-03-01 00:00:00,2000-03-01 00:00:00


In [166]:
test_old_info_df[:282245].tail(10)  # 282239

index,sample_id,location,month,day,seconds,tick,year,timestamp,no_year_timestamp
u32,str,u32,i64,i64,i64,i64,i32,datetime[μs],datetime[μs]
282235,"""test_404451""",379,12,31,72000,60,9,0009-12-31 20:00:00,2000-12-31 20:00:00
282236,"""test_305312""",380,12,31,72000,60,9,0009-12-31 20:00:00,2000-12-31 20:00:00
282237,"""test_638294""",381,12,31,72000,60,9,0009-12-31 20:00:00,2000-12-31 20:00:00
282238,"""test_492038""",382,12,31,72000,60,9,0009-12-31 20:00:00,2000-12-31 20:00:00
282239,"""test_384591""",383,12,31,72000,60,9,0009-12-31 20:00:00,2000-12-31 20:00:00
282240,"""test_20748""",0,1,1,21600,18,10,0010-01-01 06:00:00,2000-01-01 06:00:00
282241,"""test_409714""",1,1,1,21600,18,10,0010-01-01 06:00:00,2000-01-01 06:00:00
282242,"""test_148301""",2,1,1,21600,18,10,0010-01-01 06:00:00,2000-01-01 06:00:00
282243,"""test_250407""",3,1,1,21600,18,10,0010-01-01 06:00:00,2000-01-01 06:00:00


In [167]:
test_old_info_df[:618629].tail(10)  # 618623

index,sample_id,location,month,day,seconds,tick,year,timestamp,no_year_timestamp
u32,str,u32,i64,i64,i64,i64,i32,datetime[μs],datetime[μs]
618619,"""test_490095""",379,12,31,72000,60,10,0010-12-31 20:00:00,2000-12-31 20:00:00
618620,"""test_256221""",380,12,31,72000,60,10,0010-12-31 20:00:00,2000-12-31 20:00:00
618621,"""test_151665""",381,12,31,72000,60,10,0010-12-31 20:00:00,2000-12-31 20:00:00
618622,"""test_465143""",382,12,31,72000,60,10,0010-12-31 20:00:00,2000-12-31 20:00:00
618623,"""test_336617""",383,12,31,72000,60,10,0010-12-31 20:00:00,2000-12-31 20:00:00
618624,"""test_364584""",0,1,1,21600,18,11,0011-01-01 06:00:00,2000-01-01 06:00:00
618625,"""test_56256""",1,1,1,21600,18,11,0011-01-01 06:00:00,2000-01-01 06:00:00
618626,"""test_441627""",2,1,1,21600,18,11,0011-01-01 06:00:00,2000-01-01 06:00:00
618627,"""test_69092""",3,1,1,21600,18,11,0011-01-01 06:00:00,2000-01-01 06:00:00


In [168]:
test_info_df

sample_id,location,month,day,seconds,tick,no_year_timestamp
str,u32,i64,i64,i64,i64,datetime[μs]
"""test_100324""",234,1,1,0,0,2000-01-01 00:00:00
"""test_101381""",46,1,1,0,0,2000-01-01 00:00:00
"""test_102391""",179,1,1,0,0,2000-01-01 00:00:00
"""test_103808""",66,1,1,0,0,2000-01-01 00:00:00
"""test_107667""",151,1,1,0,0,2000-01-01 00:00:00
…,…,…,…,…,…,…
"""test_93387""",33,12,31,57600,48,2000-12-31 16:00:00
"""test_94307""",40,12,31,57600,48,2000-12-31 16:00:00
"""test_96792""",355,12,31,57600,48,2000-12-31 16:00:00


## old との類似度を見て近いやつを取る

In [11]:
%cd /kaggle/working

import os

from hydra import compose, initialize
from omegaconf import OmegaConf

with initialize(version_base=None, config_path="../preprocess/make_val_test_sim6"):
    cfg = compose(
        config_name="config.yaml", overrides=["debug=True"], return_hydra_config=True
    )
    print(OmegaConf.to_yaml(cfg.exp))

/kaggle/working
seed: 7
topk: 3
chunk_size: 100
output_dir: input/sim6_data
scale_dir: output/preprocess/normalize_009_rate_feat/bolton
test_data_path: input/test.parquet
test_old_data_path: input/test_old.parquet
data_dir: input/ClimSim_low-res
valid_year_id: 0



In [12]:
import json
import os
import pickle
import shutil
import sys
import time
from glob import glob
from pathlib import Path
from typing import Literal

import hydra
import numpy as np
import polars as pl
import torch
from hydra.core.hydra_config import HydraConfig
from omegaconf import DictConfig, OmegaConf
from torch.nn.functional import normalize
from torch.utils.data import DataLoader, TensorDataset
from tqdm.auto import tqdm


def compute_cosine_similarity(tensor1, tensor2):
    return torch.mm(tensor1, tensor2.t())


def get_top_k_similar_rows(matrix, target_matrix, k=5, chunk_size=1000, device="cpu"):
    matrix = matrix.to(device)
    normalized_matrix = normalize(matrix, p=2, dim=1)
    target_matrix = target_matrix.to(device)
    normalized_target_matrix = normalize(target_matrix, p=2, dim=1)

    top_k_indices = torch.empty((matrix.size(0), k), dtype=torch.long, device="cpu")

    dataset = TensorDataset(normalized_matrix)
    dataloader = DataLoader(dataset, batch_size=chunk_size)

    for ci, chunk in enumerate(tqdm(dataloader)):
        chunk_tensor = chunk[0]
        chunk_size_actual = chunk_tensor.size(0)

        cosine_similarities = compute_cosine_similarity(
            chunk_tensor, normalized_target_matrix
        ).to(device)
        top_k = torch.topk(cosine_similarities, k, dim=1)
        top_k_indices[ci * chunk_size : ci * chunk_size + chunk_size_actual] = (
            top_k.indices.cpu()
        )

    return top_k_indices

In [13]:
test_df = pl.read_csv("input/leap-atmospheric-physics-ai-climsim/test.csv")
test_old_df = pl.read_csv("input/leap-atmospheric-physics-ai-climsim/test_old.csv")

In [130]:
base_array = test_df[:, 1:].to_numpy()
old_array = test_old_df[:, 1:].to_numpy()

# scaling
feat_mean_dict = pickle.load(
    open(
        Path(cfg.exp.scale_dir) / "x_mean_feat_dict.pkl",
        "rb",
    )
)
feat_std_dict = pickle.load(
    open(
        Path(cfg.exp.scale_dir) / "x_std_feat_dict.pkl",
        "rb",
    )
)

data = torch.tensor(base_array[:, :556])
data = (data - feat_mean_dict["base"]) / (feat_std_dict["base"] + 1e-60)
data = data.float()
data_old = torch.tensor(old_array[:, :556])
data_old = (data_old - feat_mean_dict["base"]) / (feat_std_dict["base"] + 1e-60)
data_old = data_old.float()

top_k_similar = get_top_k_similar_rows(
    data,
    data_old,
    k=cfg.exp.topk,
    chunk_size=cfg.exp.chunk_size,
    device="cuda",
)

In [139]:
test_info_top1_df = test_info_df.with_columns(
    pl.Series(name="top1_index", values=top_k_similar[:, 0].numpy())
)

In [140]:
test_info_top1_df = test_info_top1_df.join(
    test_old_info_df.with_row_index("top1_index").with_columns(
        pl.col("top1_index").cast(pl.Int64)
    ),
    on="top1_index",
    suffix="_old",
).sort(["month", "day", "tick", "location"])

In [141]:
group_list = ["month", "day", "tick", "location"]

test_info_top1_df = test_info_top1_df.with_columns(
    pl.max("timestamp").over(group_list).alias("max_timestamp"),
    pl.min("timestamp").over(group_list).alias("min_timestamp"),
    pl.count("timestamp").over(group_list).alias("count"),
).with_columns((pl.col("max_timestamp") - pl.col("min_timestamp")).alias("time_diff"))

In [176]:
test_info_top1_df = test_info_top1_df.with_columns(
    pl.col("time_diff").dt.total_days().alias("time_diff_days")
).with_columns(
    ((360 < pl.col("time_diff_days")) & (pl.col("time_diff_days") < 370)).alias("is_ok")
)

In [179]:
test_info_top1_df.filter(pl.col("count") >= 2)['is_ok'].sum()

391942

In [178]:
test_info_top1_df.filter(pl.col("count") >= 2)

(409040, 22)